<a href="https://colab.research.google.com/github/soham7707/DeepSectorAI/blob/main/GRU_GARCH_BETA__SectoralAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 13.3 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib


In [3]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2022-01-01"
end_date = "2022-06-30"

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-2128990914.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-2128990914.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-2128990914.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-2128990914.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-2128990914.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [5]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [6]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [7]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [8]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2022-01-03,693.454529,1449.202637,741.773987,1817.761719
2022-01-04,706.201416,1457.689941,749.679565,1845.808472
2022-01-05,723.877563,1492.307129,764.423828,1915.103638
2022-01-06,727.263428,1468.370850,761.513855,1884.964600
2022-01-07,727.562134,1478.670166,769.467957,1897.269409
...,...,...,...,...
2022-06-23,626.733093,1288.059692,678.189209,1677.926392
2022-06-24,631.363892,1306.052002,692.060486,1692.173950
2022-06-27,634.450867,1308.415527,695.843506,1685.349121


In [9]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2022-01-04,0.018215,0.005839,0.010601,0.015311
2022-01-05,0.024722,0.023470,0.019477,0.036854
2022-01-06,0.004666,-0.016170,-0.003814,-0.015863
2022-01-07,0.000411,0.006990,0.010391,0.006507
2022-01-10,0.016561,0.005531,0.021821,0.022689
...,...,...,...,...
2022-06-23,0.004060,0.003677,0.018114,0.008587
2022-06-24,0.007362,0.013872,0.020247,0.008455
2022-06-27,0.004877,0.001808,0.005451,-0.004041


In [10]:
pip install arch


In [11]:
from arch import arch_model
import pandas as pd
import numpy as np


In [12]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [13]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...
Estimating GARCH volatility for Banking sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003374. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003659. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Pharma sector...
Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002738. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003046. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

In [14]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2022-06-23  0.017310       0.017990  0.014771
2022-06-24  0.016514       0.018030  0.014771
2022-06-27  0.015483       0.018068  0.014771
2022-06-28  0.015254       0.018030  0.014771
2022-06-29  0.014115       0.017986  0.014771


LSTM

In [15]:
pip install numpy pandas scikit-learn tensorflow


In [16]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [17]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import GRU, Dense



def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model





In [18]:
def predict_gru_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)  # same prepare function works
    model = build_gru_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [19]:
gru_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting GRU volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_gru_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    gru_volatility_data[sector] = sector_vols


Predicting GRU volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step


In [20]:
print(gru_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-06-23    0.003254  0.005190  0.004042  0.004740
2022-06-24    0.002993  0.005140  0.004127  0.004720
2022-06-27    0.002845  0.005149  0.004269  0.004723
2022-06-28    0.002844  0.004927  0.004329  0.004629
2022-06-29    0.002699  0.004712  0.004293  0.004430


In [21]:
gru_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-02-02         NaN       NaN       NaN       NaN
2022-02-03         NaN       NaN       NaN       NaN
2022-02-04         NaN       NaN       NaN       NaN
2022-02-07         NaN       NaN       NaN       NaN
2022-02-08         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2022-06-23    0.003254  0.005190  0.004042  0.004740
2022-06-24    0.002993  0.005140  0.004127  0.004720
2022-06-27    0.002845  0.005149  0.004269  0.004723
2022-06-28    0.002844  0.004927  0.004329  0.004629
2022-06-29    0.002699  0.004712  0.004293  0.004430

[101 rows x 4 columns]>

In [22]:
gru_volatility_data['IT'].head()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2022-02-02,NaN,NaN,NaN,NaN
2022-02-03,NaN,NaN,NaN,NaN
2022-02-04,NaN,NaN,NaN,NaN
2022-02-07,NaN,NaN,NaN,NaN
2022-02-08,NaN,NaN,NaN,NaN


In [23]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


/tmp/ipython-input-2331807048.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [24]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [25]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...


In [26]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT          0.93574  1.029992  0.881973  1.100392     0.000000     0.000000   
Banking     0.00000  0.000000  0.000000  0.000000     1.025734     1.076282   
Pharma      0.00000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto        0.00000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG        0.00000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT           0.000000      0.000000  0.000000    0.000000      0.000000   
Banking      1.004769      1.003439  0.000000    0.000000      0.000000   
Pharma       0.000000      0.000000  0.378681    0.732037      0.635612   
Auto         0.000000      0.000000  0.000000    0.000000      0.000000   
FMCG         0.000000      0.000000  0.000000    0.000000      0.000000   

           M&M.NS  MARUTI.NS  TATAMOTORS.NS  DABUR.NS  HINDUNILVR.NS    IT

Computing Sharpe Ratio

In [27]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [28]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            gru_mean = gru_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, gru_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(
    features,
    columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'GRU_Vol', 'Sharpe']
)


In [29]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'GRU_Vol', 'Sharpe'])


In [30]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [ -0.14687915 -13.487704    62.26793636]
Intercept: 0.1252253772895008


In [31]:
features_df['Risk_Score'] = X @ weights


In [32]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
3    WIPRO.NS   -0.269979
0  HCLTECH.NS   -0.232872
2      TCS.NS   -0.186381

Top 3 safest stocks in Banking:
          Stock  Risk_Score
7  KOTAKBANK.NS   -0.227184
5   HDFCBANK.NS   -0.210964
6  ICICIBANK.NS   -0.169563

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS   -0.168159
9     DRREDDY.NS   -0.160572
8       CIPLA.NS   -0.106248

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.202931
12      MARUTI.NS   -0.107009
11         M&M.NS   -0.094459

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
15  HINDUNILVR.NS   -0.171052
16         ITC.NS   -0.074817
14       DABUR.NS   -0.066812


GRU AND GARCH ONLY

In [33]:
from sklearn.linear_model import LinearRegression

X_new = features_df[['GARCH_Vol', 'GRU_Vol']]
y_new = features_df['Sharpe']

model_new = LinearRegression().fit(X_new, y_new)
weights_new = model_new.coef_
intercept_new = model_new.intercept_

print("New Weights (GARCH, GRU):", weights_new)


New Weights (GARCH, GRU): [-26.24051557  64.60105599]


In [34]:
features_df['Risk_Score_GRU_GARCH'] = X_new @ weights_new


In [35]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + GRU only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_GARCH')
          .head(3)[['Stock', 'Risk_Score_GRU_GARCH']])



Safest stocks in IT (GARCH + GRU only):
        Stock  Risk_Score_GRU_GARCH
3    WIPRO.NS             -0.359566
0  HCLTECH.NS             -0.326664
2      TCS.NS             -0.255944

Safest stocks in Banking (GARCH + GRU only):
          Stock  Risk_Score_GRU_GARCH
7  KOTAKBANK.NS             -0.302178
5   HDFCBANK.NS             -0.289666
6  ICICIBANK.NS             -0.223366

Safest stocks in Pharma (GARCH + GRU only):
           Stock  Risk_Score_GRU_GARCH
10  SUNPHARMA.NS             -0.273863
9     DRREDDY.NS             -0.267872
8       CIPLA.NS             -0.241713

Safest stocks in Auto (GARCH + GRU only):
            Stock  Risk_Score_GRU_GARCH
13  TATAMOTORS.NS             -0.296709
12      MARUTI.NS             -0.221213
11         M&M.NS             -0.185111

Safest stocks in FMCG (GARCH + GRU only):
            Stock  Risk_Score_GRU_GARCH
15  HINDUNILVR.NS             -0.272187
14       DABUR.NS             -0.163912
16         ITC.NS             -0.163195


GRU AND BETA ONLY

In [36]:
from sklearn.linear_model import LinearRegression

X_lb = features_df[['Beta', 'GRU_Vol']]
y_lb = features_df['Sharpe']

model_lb = LinearRegression().fit(X_lb, y_lb)
weights_lb = model_lb.coef_
intercept_lb = model_lb.intercept_

print("Weights: Beta =", weights_lb[0], ", GRU =", weights_lb[1])
print("Intercept:", intercept_lb)


Weights: Beta = -0.24662243042733 , GRU = 52.569798417690336
Intercept: 0.003424736043750598


In [37]:
features_df['Risk_Score_GRU_Beta'] = X_lb @ weights_lb


In [38]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (BiLSTM + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_Beta')
          .head(3)[['Stock', 'Risk_Score_GRU_Beta']])



Safest stocks in IT (BiLSTM + Beta only):
        Stock  Risk_Score_GRU_Beta
3    WIPRO.NS            -0.133071
0  HCLTECH.NS            -0.099679
2      TCS.NS            -0.082361

Safest stocks in Banking (BiLSTM + Beta only):
          Stock  Risk_Score_GRU_Beta
7  KOTAKBANK.NS            -0.110866
5   HDFCBANK.NS            -0.091772
6  ICICIBANK.NS            -0.079889

Safest stocks in Pharma (BiLSTM + Beta only):
           Stock  Risk_Score_GRU_Beta
10  SUNPHARMA.NS            -0.037434
9     DRREDDY.NS            -0.027443
8       CIPLA.NS             0.039763

Safest stocks in Auto (BiLSTM + Beta only):
            Stock  Risk_Score_GRU_Beta
13  TATAMOTORS.NS            -0.061517
11         M&M.NS             0.021031
12      MARUTI.NS             0.030628

Safest stocks in FMCG (BiLSTM + Beta only):
            Stock  Risk_Score_GRU_Beta
15  HINDUNILVR.NS            -0.041166
16         ITC.NS             0.027963
14       DABUR.NS             0.044568


GARCH AND BETA ONLY

In [39]:
from sklearn.linear_model import LinearRegression

X_gb = features_df[['Beta', 'GARCH_Vol']]
y_gb = features_df['Sharpe']

model_gb = LinearRegression().fit(X_gb, y_gb)
weights_gb = model_gb.coef_
intercept_gb = model_gb.intercept_

print("Weights: Beta =", weights_gb[0], ", GARCH =", weights_gb[1])
print("Intercept:", intercept_gb)


Weights: Beta = -0.1887986556237093 , GARCH = 8.790993379304563
Intercept: -0.022738815443356514


In [40]:
features_df['Risk_Score_GARCH_Beta'] = X_gb @ weights_gb


In [41]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta only):
      Stock  Risk_Score_GARCH_Beta
3  WIPRO.NS              -0.030351
1   INFY.NS              -0.026491
2    TCS.NS              -0.025129

Safest stocks in Banking (GARCH + Beta only):
          Stock  Risk_Score_GARCH_Beta
6  ICICIBANK.NS              -0.045741
5   HDFCBANK.NS              -0.034663
7  KOTAKBANK.NS              -0.031975

Safest stocks in Pharma (GARCH + Beta only):
           Stock  Risk_Score_GARCH_Beta
9     DRREDDY.NS               0.014561
10  SUNPHARMA.NS               0.020869
8       CIPLA.NS               0.064301

Safest stocks in Auto (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
13  TATAMOTORS.NS              -0.061863
11         M&M.NS              -0.019926
12      MARUTI.NS               0.001918

Safest stocks in FMCG (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
15  HINDUNILVR.NS               0.004756
16         ITC.NS               0.013236
14       DABUR.NS               0.

In [42]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_GRU_GARCH,Risk_Score_GRU_Beta,Risk_Score_GARCH_Beta
0,HCLTECH.NS,-0.232872,-0.326664,-0.099679,-0.013258
1,INFY.NS,-0.147549,-0.230138,-0.033293,-0.026491
2,TCS.NS,-0.186381,-0.255944,-0.082361,-0.025129
3,WIPRO.NS,-0.269979,-0.359566,-0.133071,-0.030351
4,AXISBANK.NS,-0.099664,-0.182484,0.008936,-0.024699


Comparison per sector

In [43]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [44]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
3,WIPRO.NS,-0.269979
0,HCLTECH.NS,-0.232872
2,TCS.NS,-0.186381



🔹 Top 3 safest stocks using Risk_Score_GRU_GARCH:


,Stock,Risk_Score_GRU_GARCH
3,WIPRO.NS,-0.359566
0,HCLTECH.NS,-0.326664
2,TCS.NS,-0.255944



🔹 Top 3 safest stocks using Risk_Score_GRU_Beta:


...

,Stock,Risk_Score_GRU_GARCH
10,SUNPHARMA.NS,-0.273863
9,DRREDDY.NS,-0.267872
8,CIPLA.NS,-0.241713



🔹 Top 3 safest stocks using Risk_Score_GRU_Beta:


,Stock,Risk_Score_GRU_Beta
10,SUNPHARMA.NS,-0.037434
9,DRREDDY.NS,-0.027443
8,CIPLA.NS,0.039763



🔹 Top 3 safest stocks using Risk_Score_GARCH_Beta:


,Stock,Risk_Score_GARCH_Beta
9,DRREDDY.NS,0.014561
10,SUNPHARMA.NS,0.020869
8,CIPLA.NS,0.064301


Comparison table

In [45]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [46]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_GRU_GARCH_Rank,Risk_Score_GRU_Beta_Rank,Risk_Score_GARCH_Beta_Rank
15,HINDUNILVR.NS,1.0,1.0,1.0,1.0
16,ITC.NS,2.0,3.0,2.0,2.0
14,DABUR.NS,3.0,2.0,3.0,3.0
